In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import gradio as gr
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense  # or GRU
from tensorflow.keras.callbacks import EarlyStopping

# 2. Load and preprocess data
df = pd.read_csv('/content/stock_data.csv')
close_prices = df['closing_price'].values.reshape(-1, 1)  # Replace 'Close' with your column name

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(close_prices)

sequence_length = 30  # Or whatever your model uses

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_prices, sequence_length)

# 3. Split into train/test (simple split)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 4. Build and train the model
model = Sequential([
    LSTM(50, return_sequences=False, input_shape=(sequence_length, 1)),  # or GRU
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stop])

# 5. Define Gradio prediction function
def predict_next_price(prices):
    try:
        # Convert input to list of floats
        if isinstance(prices, list):
            prices = [float(p[0]) if isinstance(p, list) else float(p) for p in prices]
        else:
            return "Input must be a list."
        if len(prices) != sequence_length:
            return f"Please enter exactly {sequence_length} prices."
        input_scaled = scaler.transform(np.array(prices).reshape(-1, 1))
        X_input = input_scaled.reshape((1, sequence_length, 1))
        pred_scaled = model.predict(X_input)
        pred_price = scaler.inverse_transform(pred_scaled)
        return float(pred_price[0][0])
    except Exception as e:
        return f"Error: {e}"

# 6. Create Gradio interface
iface = gr.Interface(
    fn=predict_next_price,
    inputs=gr.Dataframe(
        headers=["Closing Price"],
        row_count=sequence_length,
        col_count=1,
        label=f"Enter the last {sequence_length} closing prices"
    ),
    outputs=gr.Number(label="Predicted Next Closing Price"),
    title="Stock Price Predictor (LSTM/GRU)",
    description=f"Input the last {sequence_length} closing prices to predict the next day's closing price."
)

# 7. Launch Gradio app
iface.launch(share=True)  # share=True gives a public link in Colab

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.0934 - val_loss: 0.0239
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0080 - val_loss: 0.0077
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0070 - val_loss: 0.0070
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0053 - val_loss: 0.0083
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0046 - val_loss: 0.0104
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0052 - val_loss: 0.0115
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0053 - val_loss: 0.0056
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0041 - val_loss: 0.0075
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0039 - val_loss: 0.0088
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0038 - val_loss: 0.0054
Epoch 11/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0034 - val_loss: 0.0080
Epoch 12/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0